In [1]:
import numpy as np
import pandas as pd
import sklearn

np.random.seed(42)
np.set_printoptions(precision=6, suppress=True)
sklearn.__version__

'1.0.1'

#### 1. Dane:

In [2]:
pogoda = ['słonecznie', 'deszczowo', 'pochmurno', 'deszczowo', 'słonecznie', 'słonecznie',
          'pochmurno', 'pochmurno', 'słonecznie']
temperatura = ['ciepło', 'zimno', 'ciepło', 'ciepło', 'ciepło', 'umiarkowanie',
               'umiarkowanie', 'ciepło', 'zimno']

spacer = ['tak', 'nie', 'tak', 'nie', 'tak', 'tak', 'nie', 'tak', 'nie']      

raw_df = pd.DataFrame(data={'pogoda': pogoda, 'temperatura': temperatura, 'spacer': spacer})
df = raw_df.copy()
df

,pogoda,temperatura,spacer
0,słonecznie,ciepło,tak
1,deszczowo,zimno,nie
2,pochmurno,ciepło,tak
3,deszczowo,ciepło,nie
4,słonecznie,ciepło,tak
5,słonecznie,umiarkowanie,tak
6,pochmurno,umiarkowanie,nie
7,pochmurno,ciepło,tak
8,słonecznie,zimno,nie


#### 2. Zbadanie zmiennej docelowej:

In [26]:
# rozkład zmiennej

taregt_dist = df['spacer'].value_counts()
taregt_dist

tak    5
nie    4
Name: spacer, dtype: int64

In [6]:
# prawdopodobieństwo

df['spacer'].value_counts() / df['spacer'].value_counts().sum()

tak    0.555556
nie    0.444444
Name: spacer, dtype: float64

#### 3. Zmienne niezależne

#### - pogoda

In [8]:
df['pogoda'].value_counts()

słonecznie    4
pochmurno     3
deszczowo     2
Name: pogoda, dtype: int64

In [46]:
df_pogoda = pd.DataFrame(data={'pogoda':['słonecznie', 'deszczowo', 'pochmurno']})
df_pogoda = df_pogoda.merge(right=df.groupby(['pogoda', 'spacer']).count().unstack()['temperatura'], left_on='pogoda', right_index=True).fillna(0)
df_pogoda['P(nie)'] = df_pogoda['nie'].apply(lambda x: f'{int(x)}/{taregt_dist[1]}')
df_pogoda['P(tak)'] = df_pogoda['tak'].apply(lambda x: f'{int(x)}/{taregt_dist[0]}')
df_pogoda[['nie', 'tak']] = df_pogoda[['nie', 'tak']].astype('int')

In [39]:
# prawdopodobieństwo nie i tak w mianowniku ma rozkład zmiennej docelowej
df_pogoda

,pogoda,nie,tak,P(nie),P(tak)
0,słonecznie,1,3,1/4,3/5
1,deszczowo,2,0,2/4,0/5
2,pochmurno,1,2,1/4,2/5


#### - temperatura

In [48]:
df_temp = pd.DataFrame(data={'temperatura':['ciepło', 'umiarkowanie', 'zimno']})
df_temp = df_temp.merge(right=df.groupby(['temperatura', 'spacer']).count().unstack()['pogoda'], left_on='temperatura', right_index=True).fillna(0)
df_temp['P(nie)'] = df_temp['nie'].apply(lambda x: f'{int(x)}/{taregt_dist[1]}')
df_temp['P(tak)'] = df_temp['tak'].apply(lambda x: f'{int(x)}/{taregt_dist[0]}')
df_temp[['nie', 'tak']] = df_temp[['nie', 'tak']].astype('int')
df_temp

,temperatura,nie,tak,P(nie),P(tak)
0,ciepło,1,4,1/4,4/5
1,umiarkowanie,1,1,1/4,1/5
2,zimno,2,0,2/4,0/5


#### 4. Musimy odpowiedzieć na pytanie:

In [60]:
pytanie = pd.DataFrame(data=[['słonecznie', 'ciepło', '?']], columns=['pogoda', 'temperatura', 'spacer'])
pytanie

,pogoda,temperatura,spacer
0,słonecznie,ciepło,?


![alt text](bayes.png "Naive Bayes")

In [74]:
p_tak = (3/5 * 4/5 * 5/9) 
p_nie = (1/4 * 1/4 * 4/9) 

p_tak, p_nie

(0.26666666666666666, 0.027777777777777776)

In [75]:
# Prawdopodobieństwo powinno być równe 1

In [90]:
# Prawdopodobieństwo, ze wyjdziemy w słoneczną pogodę i ciepłą temp. wyosi 0,9057
# Prawdopodobieństwo, ze  nie wyjdziemy w słoneczną pogodę i ciepłą temp. wyosi 0,0943

# normalizacja wcześniej uzyskanych wyników 
P_tak = p_tak / (p_tak + p_nie)
P_nie = p_nie / (p_tak + p_nie)

P_tak, P_nie

(0.8727272727272728, 0.12727272727272726)

### 5. Zakładamy, że rozkład jest normalny

![alt text](normal_dist_bayes.png "Gaussian Naive Bayes classifier")